# 变分推断  

## 背景  
  
  为什么需要变分推断，在上节讲过了推断。 
  
 推断： $$P(x_F|x_E)=\frac{P(x_E,x_F)}{p(x_E)}=\frac{P(x_E,x_F)}{\sum_{x_F}p(x_E,x_F)}$$   
   
 推断的抽象（贝叶斯）：X是已知样本，$\theta$是X分布的参数，$\hat{X}$是新样本   
   
 $$P(\theta|X)=\frac{P(\theta ，X)}{p(X)}=\frac{P(X|\theta)P(\theta)}{p(X)}=\frac{P(X|\theta)P(\theta)}{\int_{\theta}P(\theta ，X)d\theta}$$   
   
这里$P(\theta|X)$是后验概率，即根据现有的样本，去调整真实样本X的真实分布，让调整后的分布，更接近X的本来分布。  
  
$P(\theta)$是先验分布，即我们现在根据以往经验所知道的X分布。（通过一次次的后验分布调整，会使得我们的先验分布接近其真实的分布）  
  
$P(X|\theta)$是似然，即根据现有的经验分布，得到样本X的概率是多少，（我们目标是极大似然）  
  
$p(X)=\int_{\theta}P(\theta ，X)d\theta$是证据。  
  
**推断就是要求这个后验概率**  
  
**当推断出这个后验概率后,基于后验概率，我们可以对新样本做出决策**   
  
  
<br>  
  
<br>  

## 变分推断  
  
  我们要求的是后验概率$P(\theta|X)$，但是这个后验概率在维度特别高的时候，或者一些情况下，是直接求不出来的。一种高效的求解后验概率$P(\theta|X)$的方法是变分推断。  
    
  变分推断：我们目标是要求$P(\theta|X)$，但是直接求这个复杂的后验概率很难，所以我们直观的想法，就是可以用一个简单分布$Q(Z)$去逼近这个复杂的分布$P(\theta|X)$(因为简单分布好算，且函数图形上与复杂分布近似。)。  
    
 现在我们取一个简单的分布$Q(Z)$这个分布的所有分量是相互独立的（够简单）。$Z=\{Z_1,Z_2,……,Z_m\} $由于相互独立，则有$Q(Z)=Q(Z_1)Q(Z_2)……Q(Z_m)$由于相互独立在计算$Q(Z)$时可分分开单独计算某个分量的分布，最后合并所有分量，就可以得到$Q(Z)$  
   
   
**现在问题转化为如何求解$Q(Z)$使得$Q(Z)$逼近$P(\theta|X)$。**  
  
直观的想法是直接求$Q(Z)$和$P(\theta|X)$的KL散度，让KL散度最小就行了。即最小化$KL(Q(Z)||P(Z|X)\;)=\int Q(Z) ln\frac{Q(Z)}{P(Z|X)}dZ$。但是要想最小化这个公式，任然是做不到的。（首先Z空间就及其大，再则即使假设了简单的分布$Q(Z)$,且其参数为$\eta$，但我们本身就是要推断出${P(Z|X)}$，所以这样是不可行的。）  
  
直接最小化$Q(Z)$和$P(\theta|X)$的KL散度我们做不到，但是我们可以一个迂回的方式，优化另外一个目标，在完成这个目标的同时，附带的结果包含了最小化$Q(Z)$和$P(\theta|X)$的KL散度。  
  
**这就是变分推断的精妙之处。**  
  
首先我们假设真实样本X，服从一个分布$P(X)$。由于服从的是X原本的真实分布，所以$P(X)$也应该是一个真实的具体的值。  
  
$P(X)$  
  
$P(X) = \frac{P(X,Z)}{P(Z|X)}$    
  
两边取对数：  $\qquad\qquad\;\; ln P(X) =ln \frac{P(X,Z)}{P(Z|X)}$   
  
$\qquad\qquad\qquad\qquad\;\;\;   ln P(X) =ln P(X,Z) - ln P(Z|X)$   
  
$\qquad\qquad\qquad\qquad\;\;\;   ln P(X) =ln P(X,Z) -ln P(Z)  - (ln P(Z|X) -ln P(Z))$    
  
$\qquad\qquad\qquad\qquad\;\;\;   ln P(X) =ln \frac{P(X,Z)}{P(Z)}   - ln \frac{P(Z|X)}{P(Z)} $  
  
对左右两边乘以$P(Z)\quad$： $\; P(Z)ln P(X) = P(Z)ln \frac{P(X,Z)}{P(Z)}  -P(Z) ln \frac{P(Z|X)}{P(Z)}$  
  
两边对Z求积分： $\qquad\quad \int P(Z)ln P(X)dZ =\int[P(Z)ln \frac{P(X,Z)}{P(Z)}  -P(Z) ln \frac{P(Z|X)}{P(Z)}]dZ$   
  
由于：$\qquad\qquad\qquad \int P(Z)ln P(X)dZ =ln P(X)$   
  
所以有：  $\qquad\qquad\quad  ln P(X) =\int[P(Z)ln \frac{P(X,Z)}{P(Z)}  -P(Z) ln \frac{P(Z|X)}{P(Z)}]dZ$    
  
$\qquad\qquad\qquad\qquad\;\;\;  ln P(X) =\underbrace{\int P(Z)ln \frac{P(X,Z)}{P(Z)}dZ}_{\text{公式一}}\underbrace{ - \int P(Z) ln \frac{P(Z|X)}{P(Z)} }_{\text{公式二（带负号）}} dZ$   
  
$\qquad\qquad\qquad\qquad\;\;\;  ln P(X) =\underbrace{\int P(Z)ln \frac{P(X,Z)}{P(Z)}dZ}_{\text{公式一}}+\underbrace{KL(P(Z)||P(Z|X))}_{公式二}$   
  
观察这个公式，我们发现$ln P(X)$是我们理想的分布数值，是真实确定值。我们发现左边是定值，所以只要极大化公式一， 就能做大$Q(Z)$和$P(\theta|X)$的KL散度最小。  
  
**关键最大化公式一：**  
  
令：  $L(Q(Z))=\int P(Z)ln \frac{P(X,Z)}{P(Z)}dZ \qquad$  可以看出这个$L(Q(Z))$就是一个泛函。所以求使这个泛函达到最大值的$Q(Z)$就是变分法。变分推断中变分就是从这里得到的。  
   
解法：  
$L(Q(Z))=\int P(Z)ln \frac{P(X,Z)}{P(Z)}dZ \qquad$  
  
$L(Q(Z))=\underbrace{\int P(Z)ln P(X,Z)dZ}_{part 1}-\underbrace{\int P(Z)ln P(Z)dZ}_{part 2}$  
  
这里我们求解这个泛函，精确值不好算，我们使用迭代的方法。即沿着使$L(Q(Z))$增大的方向，改变Q(Z）  
    
这里我们取一个简单的分布$Q(Z)$这个分布的所有分量是相互独立的（够简单）。$Z=\{Z_1,Z_2,……,Z_m\} $。我们的思想是：使一个分量$Z_i$作为变量，其余分量$Z_{i\neq j}$ 的分布都固定，只求一个分量$Z_i$分布，减少参数，更简单。**这样在一次迭代中，我们分M次求出这m个分量的分布，最后可以组合出本次迭代$Q^t(Z)$分布。然后我们把本次迭代的$Q^t(Z)$分布，传入下次迭代，即下次迭代时，使一个分量$Z_i$作为变量，其余分量$Z^t_{i\neq j}$ 的分布都固定,这里固定的就是为上次迭代时得到的Z分量分布。  
$\color{red}{\text{所以在每次迭代时，我们都是沿着使L(Q(Z))增大的方向移动，在多次迭代后泛函L(Q)趋于稳定之后，是Q使泛函达到最大。}}$ 
$\color{red}{\text{泛函达到最大化，则KL散度最小。所以这个Q就是逼近后验概率P(Z|X)的简单分布。}}$**   
  
**现在分部求解$L(Q(Z))$**    
  
 part1（固定除i之外其他的分量）:  
 $\quad \int P(Z)ln P(X,Z)dZ \;=\; \int_{dz_i}P(Z_i)\underbrace{[ …… \int_{dz_m} P(Z_1)……P(Z_m)ln P(X,Z) dZ_1……]}_{\text{m-1个期望，是一个常数}}dZ_i \;=\; \int_{dz_i}P(Z_i)\{E_{i\neq j}[ln P(X,Z)]\}dZ_i$   
   
 part2（固定除i之外其他的分量）:  
   
 $\quad \int P(Z)ln P(Z)dZ = \int_{dz_1}…… \int_{dz_m}P(Z_1)……P(Z_m)[ln P(Z_1)+……+ln P(Z_m)]dZ_1……dZ_m$  
   
 $\qquad\qquad\qquad\quad = \int_{dz_1}P(Z_1)ln P(Z_1)dZ_1+…… +\int_{dz_m}P(Z_m)ln P(Z_m)dZ_m$  
   
$\qquad\qquad\qquad\quad = \int_{dz_i}P(Z_i)ln P(Z_i)dZ_i+const$   
    
<br>     
      
所以$L(Q(Z_i))=part1-part2=\; \int_{dz_i}P(Z_i)\{E_{i\neq j}[ln P(X,Z)]\}dZ_i - \int_{dz_i}P(Z_i)ln P(Z_i)dZ_i - const$   
  
$\qquad\qquad\qquad\qquad\qquad\quad = \; \int_{dz_i}P(Z_i)\{E_{i\neq j}[ln P(X,Z)]\} - P(Z_i)ln P(Z_i) dZ_i - const$   
  
$\qquad\qquad\qquad\qquad\qquad\quad = \; \int_{dz_i}P(Z_i)ln \frac{\{E_{i\neq j}[ln P(X,Z)]\}}{P(Z_i)}   dZ_i - const$   
  
$\qquad\qquad\qquad\qquad\qquad\quad = \; - KL(\{E_{i\neq j}[ln P(X,Z)]\}||P(Z_i))    - const$   
  
**最总的泛函$L(Q(Z_i))$**   
  
 $L(Q(Z_i)) = \; - KL(\{E_{i\neq j}[ln P(X,Z)]\}||P(Z_i))    - const$  
   
 由此式可以知道要使此泛函达到最大值则只需要$ KL(\{E_{i\neq j}[ln P(X,Z)]\}||P(Z_i))$这个KL散度为0即可。而概率分布$\{E_{i\neq j}[ln P(X,Z)]\}$ 是常数已知的。所以只要 $Q(Z_i)=\{E_{i\neq j}[ln P(X,Z)]\}$即可让这个KL散度最小。    
   
<br>  

<br>
   
**所以在一次迭代求解泛函的过程是**  
  
  $Q(Z_1)=\{E_{j\neq 1}[ln P(X,Z)]\}$  
    
  $Q(Z_2)=\{E_{j\neq 2}[ln P(X,Z)]\} \qquad Q(Z_1)\text{也要用上面最新的}$  
    
  ^$…………$  
    
  $Q(Z_m)=\{E_{j\neq m}[ln P(X,Z)]\}$  
    
  这样可以求出使泛函达到最大的m个$Q(Z_i)$分布。而这些是相互独立的，所以本次迭代有$Q(Z)=Q(Z_1)……Q(Z_m)$   
    
<br>  

对上述单次迭代的过程，进行多次迭代，（每次迭代的Q(Z)传入下次迭代，在下次迭代过程中，固定的值就是上次的结果）。经过多次迭代，ELOB趋于稳定，$Q(Z)$逼近后验概率$P（Z|X）$  
  
<br>  
  
<br>  
Q（Z）分布随迭代次数的变化
![](img/1.PNG)  
    
 <br>  
  
<br>    
    
![](img/2.PNG)    
  
 <br>  
  
<br> 

 <br>  
  
<br> 

# 随机梯度变分推断  （涉及重参数技巧）
   
  **上述的变分推断，是经典的变分推断。但是这种变分推断的适用性并不好，因为1、假设$Z_1,Z_2,……，Z_m$相互独立，这个假设太强了。2、即使假设对一些问题成立了，但最后迭代计算$Q(Z_i)$，计算积分时，一些简单的是可以计算的，但复杂情况，任然有可能是不可计算的。所以有了随机梯度变分推断 **   
    
 现在我们的想法就是使用梯度下降法来求变分推断。  
   
   在上一节中，我们要让$Q(Z)$逼近后验概率分布$P(Z|X)$，只需最大化ELOB公式即可。即最大化这个公式$L(Q(Z))=\int P(Z)ln \frac{P(X,Z)}{P(Z)}dZ \qquad$  
     
   在上一节中，我们是用一点点的坐标上升法做的。现在我们使用随机梯度下降法来做。  
      
$$L(p_{\phi})=E_{p_{\phi}(Z)}[lnP(X,Z) -ln p_{\phi}(Z) ]$$  
  
这里$Q(Z)$是一参数为$\phi$的分布。我们要使用梯度下降法求出使用这个期望最大的。分布$Q(Z)$。那么我们现在只需要在这个公式上对参数$\phi$求梯度，就可以了。  
  
  $$\nabla_{\phi}L(p_{\phi})=\nabla_{\phi} E_{p_{\phi}(Z)}[lnP(X,Z) -ln p_{\phi}(Z) ]$$  
    
   $$\nabla_{\phi}L(p_{\phi})=\nabla_{\phi} \int_z p_{\phi}(Z)[lnP(X,Z) -ln p_{\phi}(Z) ]dz$$   
     
**积分和求导的顺序可以调换**  
  
 $$\nabla_{\phi}L(p_{\phi})=\int_z \nabla_{\phi}  \{p_{\phi}(Z)[lnP(X,Z) -ln p_{\phi}(Z) ]\}dz$$   
   
  $$\nabla_{\phi}L(p_{\phi})=\underbrace{\int_z \nabla_{\phi}  p_{\phi}(Z)\{[lnP(X,Z) -ln p_{\phi}(Z) ]\}dz}_{A} \;\;+\;\; \underbrace{\int_z p_{\phi}(Z)\nabla_{\phi}[lnP(X,Z) -ln p_{\phi}(Z) ]dz}_{B}$$   
    
$B=\int_z p_{\phi}(Z)\nabla_{\phi}[lnP(X,Z) -ln p_{\phi}(Z) ]dz $  
  
$\quad=\int_z- p_{\phi}(Z)\frac{1}{p_{\phi}(Z)} \nabla_{\phi}p_{\phi}(Z)dz $  
  

$\quad=-\nabla_{\phi}\int_z  p_{\phi}(Z)dz $    
  
$\quad=0$  
  
<br>  
  
$A=\int_z \nabla_{\phi}  p_{\phi}(Z)\{[lnP(X,Z) -ln p_{\phi}(Z) ]\}dz$  
  
$\quad=\int_z   p_{\phi}(Z)\times [\nabla_{\phi} lnp_{\phi}(Z)] \times\{[lnP(X,Z) -ln p_{\phi}(Z) ]\}dz$  
  
$\quad=E_{p_{\phi}(Z)} [  \nabla_{\phi} lnp_{\phi}(Z)\{[lnP(X,Z) -ln p_{\phi}(Z) ]\}]$  

<br>
    
所以：  
  
 $\nabla_{\phi}L(p_{\phi})=E_{p_{\phi}(Z)} [  \nabla_{\phi} lnp_{\phi}(Z)\{[lnP(X,Z) -ln p_{\phi}(Z) ]\}]$   
   
<br>  
  
**所以利用梯度下降法,时对参数$\phi$求的梯度就是上式了。$\color{red}{此时可以用采样法计算出参数的梯度。就可以使用梯度下降法了}$**  
  
**但是一个问题是：$\nabla_{\phi} lnp_{\phi}(Z)$中ln的梯度变化非常大，假如采样一个z后q（z）接近0，另一个z的q（z）接近1，这样对数lnq（z）的梯度相差非常大，所以会存在高方差的问题，高方差问题会导致在梯度更新时不稳定。**   
  
**$\color{red}{所以采用重参数技巧来降低方差，使得梯度能稳定的更新}$**   
  
<br>  

要是z不从$q_φ(z)$中直接采样，而是从一个已知的无参数的分布$p_(ε)$中采样得到ε，再通过一个变换$z=g_{\phi}(ε)$得到z，通过这样的过程来采样z，将z的随机性转化为ε的随机性，这样就消除了高方差的问题.  
  
已知：$ε\text{~}p_(ε) \qquad z=g_φ(ε)$  
  
那么：   
  
$\nabla_{\phi}L(p_{\phi})=\nabla_{\phi} E_{p_{\phi}(Z)}[lnP(X,Z) -ln p_{\phi}(Z) ]$  
  
$\qquad\quad\;\;=\nabla_{\phi} E_{ε\text{~}p_(ε)}[lnP(X,Z) -ln p_{\phi}(Z) ] \qquad\qquad \text{Z没有随机性了}$  
  
$\qquad\quad\;\;=E_{ε\text{~}p_(ε)}[\nabla_{\phi} (lnP(X,g_φ(ε)) -ln p_{\phi}(g_φ(ε)) )]$  
  
$\qquad\quad\;\;=E_{ε\text{~}p_(ε)}[\nabla_{Z} (\underbrace{lnP(X,g_φ(ε)) -ln p_{\phi}(g_φ(ε)) )}_{f}]*[\nabla_{\phi} Z] \qquad\qquad \text{链式法则} \frac{\partial f}{\partial ε}=\frac{\partial f}{\partial Z}* \frac{\partial Z}{\partial ε} $  
  
$\qquad\quad\;\;=E_{ε\text{~}p_(ε)}[\nabla_{Z}f\nabla_{ε}g_φ(ε)] $  
  
这样我们从无参数的$ε\text{~}p_(ε) $ 采样大量$ε$就行。带入上式，就可以求出**低方差的梯度。**   
  
<br>  

使用梯度下降法：
    
$$\phi^{t+1}= \phi^{t}+\lambda \nabla_{\phi}L(p_{\phi})$$